In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
from posarmctools.readdata import *
%load_ext autoreload
%autoreload 2

In [3]:
import sys

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [5]:
data_date = "2017_10_12_11_47_05"
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_12/" + data_date

In [6]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters( params_filename )

In [8]:
blocksPerFile = params.blocksPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "blocksPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    blocksPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 0
nbFiles = 67
lastFile = firstFile + nbFiles - 1
firstBlock = firstFile * blocksPerFile

blocksPerFile = 6, samplesPerRamp = 6000, rampsPerFile = 1500


In [9]:
lastFile

66

### Read the data

In [10]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

adc_A = np.zeros( (nbFiles, samplesPerFile) )
adc_B = np.zeros( (nbFiles, samplesPerFile) )

In [11]:
lastBlock = (nbFiles-1) * blocksPerFile + firstBlock
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * blocksPerFile + firstBlock) )
    filename = data_dir + "/record" + nb + ".bin"
    readFile( filename, samplesPerFile, timeSerie_A, timeSerie_B )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBlock ))
    
    adc_A[ k, : ] = timeSerie_A
    adc_B[ k, : ] = timeSerie_B

k = 0, block = 0 / 396
k = 1, block = 6 / 396
k = 2, block = 12 / 396
k = 3, block = 18 / 396
k = 4, block = 24 / 396
k = 5, block = 30 / 396
k = 6, block = 36 / 396
k = 7, block = 42 / 396
k = 8, block = 48 / 396
k = 9, block = 54 / 396
k = 10, block = 60 / 396
k = 11, block = 66 / 396
k = 12, block = 72 / 396
k = 13, block = 78 / 396
k = 14, block = 84 / 396
k = 15, block = 90 / 396
k = 16, block = 96 / 396
k = 17, block = 102 / 396
k = 18, block = 108 / 396
k = 19, block = 114 / 396
k = 20, block = 120 / 396
k = 21, block = 126 / 396
k = 22, block = 132 / 396
k = 23, block = 138 / 396
k = 24, block = 144 / 396
k = 25, block = 150 / 396
k = 26, block = 156 / 396
k = 27, block = 162 / 396
k = 28, block = 168 / 396
k = 29, block = 174 / 396
k = 30, block = 180 / 396
k = 31, block = 186 / 396
k = 32, block = 192 / 396
k = 33, block = 198 / 396
k = 34, block = 204 / 396
k = 35, block = 210 / 396
k = 36, block = 216 / 396
k = 37, block = 222 / 396
k = 38, block = 228 / 396
k = 39, block =

In [12]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)
B_reshaped = adc_B.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [13]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

**Set the shifted flag properly**

In [14]:
shifted = 0

In [ ]:
ifft_a = np.fft.ifft( (A_reshaped[::,0:3000]), axis = 1 )

In [ ]:
plt.figure()
vmin = -15
vmax = 7
nbPointsInRange = 500
plt.plot(  20 * np.log10( np.abs( ifft_a[2500,0:nbPointsInRange] ) ) )
plt.plot( np.ones( nbPointsInRange ) * vmin, "b" )
plt.plot( np.ones( nbPointsInRange ) * vmax, "r" )

In [ ]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,100:500] ) ), 
            aspect='auto', vmin=-15, vmax=7, cmap='jet' )
plt.title("Track 1 - record 2017_10_12_11_47_05\nFiles {} to {}".format(firstFile, lastFile) )

## Remove coupling from raw data

In [ ]:
coupling_A = np.average( A_reshaped, 0 )
A2 = A_reshaped - coupling_A

## Build RD

In [15]:
RDc = build_rd_from_data( params, A_reshaped, shifted )

shape of the samples matrix = (100500, 6000)
Data are NOT shifted


In [ ]:
coupling_RD = np.average(RDc, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD )

In [ ]:
RD2 = RDc - coupling_RD

In [ ]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [ ]:
np.save(data_dir + '/RD_files_{}_{}'.format(firstFile, lastFile), RDc)

In [ ]:
ifft_RD = np.fft.ifft(RDc, axis = 1)
print('ifft_RD done')

In [ ]:
nbPointsInRange = 300
x0 = 0
x1 = x0 + nbPointsInRange
y0 = 0
y1 = 0
extent = [ x0, x1, y0, y1 ]
rangeStart = 0
rangeStop = 500
azStart = 6000
azStop = 15000

In [ ]:
plt.matshow( 20 * np.log10( np.abs( ifft_RD[:, rangeStart:rangeStop] ) ), aspect='auto')
plt.title(data_date)
ax = plt.gca()
plt.colorbar()

In [ ]:
fft2d = np.fft.fftshift( np.fft.fft( ifft_RD, axis = 0 ) , axes = 0)
print('fft2d done')

In [ ]:
plt.matshow( 20 * np.log( np.abs( fft2d[:, rangeStart:rangeStop] ) ), aspect='auto', cmap='jet')
plt.title(data_date)
ax = plt.gca()
plt.colorbar()

In [ ]:
adc_A_all = adc_A.reshape( rampsPerFile * nbFiles * samplesPerRamp )

In [ ]:
plt.figure()
ramp = 10
shift = 38
sampleStart = ramp * samplesPerRamp
plt.plot( adc_A_all[sampleStart : sampleStart+6000 ], 'k',
         label='both ramps' )
plt.plot( adc_A_all[sampleStart : sampleStart+3000 ],
         label='up ramp' )
plt.plot( np.flipud( adc_A_all[ sampleStart+3000 : sampleStart+6000 ] ),
         label='flipped down ramp' )
plt.plot( np.flipud( adc_A_all[ (sampleStart+3000+shift) : (sampleStart+6000+shift) ] ),
         label='flipped down ramp SHIFTED' )
plt.grid()
plt.legend()

In [ ]:
plt.figure()
ramp = 0
plt.plot( A2[ramp, 0 : 6000 ], 'k', label='both ramps' )
plt.plot( A2[ramp, 0 : 3000 ], label='up ramp' )
plt.plot( np.flipud( A2[ramp, 3000 : 6000 ] ), label='flipped down ramp' )
plt.grid()
plt.legend()
plt.title('A2')

In [ ]:
params.samplesPerRamp

In [ ]:
shifted

In [ ]:
import scipy.signal as signal

In [ ]:
s1 = A_reshaped[ramp, 0 : 3000 ]
s2 = np.flipud( A_reshaped[ramp, 4000 : 5500 ] )

corr = signal.correlate( s1, s2, mode='valid' )

plt.figure()

plt.subplot(211)
plt.plot( corr, '.' )
plt.grid()

plt.subplot(212)
plt.plot( s1 )
plt.plot( s2 )
plt.plot( np.arange(1500) + 538, s2 )
plt.grid()

plt.suptitle( 'max = {}'.format( np.where( corr == np.max(corr) )[0] ) )

In [ ]:
plt.plot(s1)